In [1]:
import subprocess
from matplotlib import pyplot as plt
%matplotlib inline
import pytesseract
from PIL import Image

In [2]:
TEXT_IMAGE = "ideal_version.jpg"
PHOTO_IMAGE = "0.jpg"
PROCESSED_IMAGE = "7.jpg"

In [3]:
def binarize(input_file_path, output_file_path="binarized_image.jpg"):
    process = subprocess.run(['./binarizewolfjolion', input_file_path, output_file_path])
    return Image.open(output_file_path)

In [4]:
def image_to_text(image_path, binarization_needed=True):
    if binarization_needed:
        image = binarize(image_path)
    else:
        image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

Можно применять разные меры схожести двух текстов:

https://stackabuse.com/levenshtein-distance-and-text-similarity-in-python/

Остановимся на Cходствe Джаро—Винклера.

In [5]:
import jellyfish

In [6]:
def get_texts_similarity(original_image_path, image_path, binarization_needed=True):
    original_text = image_to_text(original_image_path, binarization_needed=False)
    text = image_to_text(image_path, binarization_needed=binarization_needed)
    return jellyfish.jaro_winkler(original_text, text) 

In [12]:
get_texts_similarity(TEXT_IMAGE, PHOTO_IMAGE, binarization_needed=False)

0.9144614022488351